Implementing SimpleCNN for braintumor detection


In [11]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import datasets, transforms
from torchvision.io import read_image
from google.colab import drive
drive.mount('/content/drive')
dataset_path='/content/drive/My Drive/Colab Notebooks/brain_tumor_dataset'

transform_training=transforms.Compose([
    transforms.Resize((240,240)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
])

dataset=datasets.ImageFolder(root=dataset_path, transform=transform_training)
dataset_size=len(dataset)
train_size=int(0.8* dataset_size)  #taking 80% data in training
test_size=dataset_size -train_size

train_dataset, test_dataset= random_split(dataset, [train_size, test_size])

batch_size = 32
train_loader=DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader=DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



#CNN model
class CNN(nn.Module):
  def __init__ (self):
    super(CNN, self).__init__()
    self.conv1=nn.Conv2d(3,32,kernel_size=3, padding=1)
    self.activation=nn.ReLU()
    self.batch_norm=nn.BatchNorm2d(32)
    self.maxpool2d=nn.MaxPool2d(kernel_size=2, stride= 4)
    self.maxpool2d_1=nn.MaxPool2d(kernel_size=2, stride=2)
    self.maxpool2d_2=nn.MaxPool2d(kernel_size=2, stride=2)
    self.flatten=nn.Flatten()
    self.dense=nn.Linear(32*15*15, 128)
    self.dropout=nn.Dropout(0.5)
    self.dense_1=nn.Linear(128,2)
    self.softmax=nn.Softmax(dim=1)

  def forward(self, x):
    x=self.conv1(x)
    x=self.activation(x)
    x=self.batch_norm(x)
    x=self.maxpool2d(x)
    x=self.maxpool2d_1(x)
    x=self.maxpool2d_2(x)
    x=self.flatten(x)
    x=self.dense(x)
    x=self.dropout(x)
    x=self.dense_1(x)
    x=self.softmax(x)
    return x

model=CNN()
loss_check=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)


# num of iterations for training
iters=10
for epoch in range(100):
  model.train()
  for batch, labels in train_loader:
    optimizer.zero_grad()
    outputs=model(batch)
    loss=loss_check(outputs, labels)
    loss.backward()
    optimizer.step()

model.eval()
correct_predictions=0
total_samples=0
with torch.no_grad():
  for batch, labels in test_loader:
    outputs= model(batch)
    _, predicted=torch.max(outputs, 1)
    total_samples+=labels.size(0)
    correct_predictions+=(predicted==labels).sum().item()

accuracy=correct_predictions/total_samples
print(f'Accuracy : {accuracy*100 : .2f}%')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Accuracy :  82.35%
